In [142]:
import math
import numpy as np
from scipy.stats import bernoulli
from torchvision.datasets import MNIST
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

In [143]:
def download_mnist(is_train: bool):
    dataset = MNIST(root='./data',
                    transform=lambda x: np.array(x).flatten() / 255.0,
                    download=True,
                    train=is_train)
    mnist_data = []
    mnist_labels = []
    for image, label in dataset:
        mnist_data.append(image)
        mnist_labels.append(label)
    return np.array(mnist_data), np.array(mnist_labels)

In [144]:
def one_hot_encode(labels):
    encoder = OneHotEncoder(sparse_output=False)
    return encoder.fit_transform(labels.reshape(-1, 1))

In [145]:
def weight_initialization_Xavier_Uniform(input_dim, output_dim):
    low_bound = -math.sqrt(6/(input_dim + output_dim))
    upper_bound = math.sqrt(6/(input_dim + output_dim))
    W = np.random.uniform(low_bound, upper_bound, size=(input_dim, output_dim))
    b = np.zeros((1, output_dim))
    return W, b

In [146]:
def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return expZ / np.sum(expZ, axis=1, keepdims=True)

In [147]:
def sigmoid(Z):
    return 1/(1 + np.exp(-Z))

In [148]:
def forward_propagation_with_dropout(X, W1, b1, W2, b2, dropout_rate=0.35):
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)

    # Implement dropout_rate% dropout
    dropouts = bernoulli.rvs(dropout_rate, size=A1.shape) 

    A1_dropout = A1 * dropouts / (1 - dropout_rate)

    Z2 = np.dot(A1_dropout, W2) + b2
    A2 = softmax(Z2)
    return A1_dropout, A2, dropouts

In [149]:
def forward_propagation_no_dropout(X, W1, b1, W2, b2):
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)

    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)
    return A1, A2

In [150]:
def compute_loss_cross_entropy(A, Y):
    m = Y.shape[0]
    log_likelihood = -np.log(A[range(m), Y.argmax(axis=1)])
    return np.sum(log_likelihood) / m

In [151]:
def backward_propagation(X, Y, A2, A1, W2, dropouts):
    m = X.shape[0]
    
    # Output layer error term (dZ2)
    dZ2 = A2 - Y  # Derivative of softmax + cross-entropy
    dW2 = np.dot(A1.T, dZ2) / m  # Gradient w.r.t. weights (output layer)
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m  # Gradient w.r.t. biases (output layer)
    
    # Hidden layer error term (dZ1)
    dZ1 = np.dot(dZ2, W2.T) * A1 * (1 - A1)  # For sigmoid activation (derivative of sigmoid)
    
    dZ1 = dZ1 * dropouts
    dZ1 /= (1 - 0.35)

    dW1 = np.dot(X.T, dZ1) / m  # Gradient w.r.t. weights (hidden layer)
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m  # Gradient w.r.t. biases (hidden layer)
    
    return dW1, db1, dW2, db2


In [152]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    
    return W1, b1, W2, b2

In [153]:
def train_model(train_X, train_Y, input_dim, intermediary_dim, output_dim, epochs=100, learning_rate=0.01, batch_size=100):
    W1, b1 = weight_initialization_Xavier_Uniform(input_dim, intermediary_dim)
    W2, b2 = weight_initialization_Xavier_Uniform(intermediary_dim, output_dim)
    
    for epoch in range(epochs):
        perm = np.random.permutation(train_X.shape[0])
        train_X = train_X[perm]
        train_Y = train_Y[perm]
        
        for i in range(0, train_X.shape[0], batch_size):
            X_batch = train_X[i:i+batch_size]
            Y_batch = train_Y[i:i+batch_size]
            
            # Forward propagation
            A1, A2, dropouts = forward_propagation_with_dropout(X_batch, W1, b1, W2, b2) 
            
            # Compute loss (optional for tracking)
            loss = compute_loss_cross_entropy(A2, Y_batch)
            
            # Backward propagation
            dW1, db1, dW2, db2 = backward_propagation(X_batch, Y_batch, A2, A1, W2, dropouts) 
            
            # Update parameters
            W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
            
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}')
    
    return W1, b1, W2, b2

In [154]:
def predict(X, W1, b1, W2, b2):
    A2 = forward_propagation_no_dropout(X, W1, b1, W2, b2)[1]
    return np.argmax(A2, axis=1)

In [155]:
train_X, train_Y = download_mnist(True)
test_X, test_Y = download_mnist(False)
    
train_Y = one_hot_encode(train_Y)
test_Y_one_hot = one_hot_encode(test_Y)
    
input_dim = 784
intermediary_dim = 100
output_dim = 10
epochs = 200
learning_rate = 0.01
batch_size = 100
    
W1, b1, W2, b2 = train_model(train_X, train_Y, input_dim, intermediary_dim,output_dim, epochs, learning_rate, batch_size)
    
train_predictions = predict(train_X, W1, b1, W2, b2)
test_predictions = predict(test_X, W1, b1, W2, b2)
    
print(f'Training Accuracy: {accuracy_score(np.argmax(train_Y, axis=1), train_predictions) * 100:.2f}%')
print(f'Test Accuracy: {accuracy_score(test_Y, test_predictions) * 100:.2f}%')

Epoch 1/200, Loss: 2.3044
Epoch 2/200, Loss: 2.0566
Epoch 3/200, Loss: 2.0213
Epoch 4/200, Loss: 1.7431
Epoch 5/200, Loss: 1.7413
Epoch 6/200, Loss: 1.6215
Epoch 7/200, Loss: 1.4943
Epoch 8/200, Loss: 1.4184
Epoch 9/200, Loss: 1.3101
Epoch 10/200, Loss: 1.2578
Epoch 11/200, Loss: 1.2551
Epoch 12/200, Loss: 1.0637
Epoch 13/200, Loss: 1.0971
Epoch 14/200, Loss: 1.0770
Epoch 15/200, Loss: 1.0309
Epoch 16/200, Loss: 0.9130
Epoch 17/200, Loss: 1.0021
Epoch 18/200, Loss: 0.9399
Epoch 19/200, Loss: 0.8998
Epoch 20/200, Loss: 0.9349
Epoch 21/200, Loss: 0.9425
Epoch 22/200, Loss: 0.7922
Epoch 23/200, Loss: 0.7923
Epoch 24/200, Loss: 0.9429
Epoch 25/200, Loss: 0.8534
Epoch 26/200, Loss: 0.8514
Epoch 27/200, Loss: 0.8476
Epoch 28/200, Loss: 0.7547
Epoch 29/200, Loss: 0.8239
Epoch 30/200, Loss: 0.8696
Epoch 31/200, Loss: 0.7410
Epoch 32/200, Loss: 0.9314
Epoch 33/200, Loss: 0.5896
Epoch 34/200, Loss: 0.7242
Epoch 35/200, Loss: 0.6228
Epoch 36/200, Loss: 0.7004
Epoch 37/200, Loss: 0.6669
Epoch 38/2